# Poznámka ke Gaussově eliminaci

Gaussova eliminace je notoricky citlivá na nepřesnosti v počítání se strojovými čísly.
Pěkně to můžeme demonstrovat na Hilbertově matici $H_n$ což je $n \times n$ matice, jejíž prvky splňují
$$
(H_n)_{ij} = \frac{1}{i + j - 1}, \quad i,j=1,\ldots,n.
$$
Takovouto matici můžeme _exaktně_ reprezentovat pomocí racionálních čísel:

In [1]:
hilbert(n) = [ 1 // (i + j - 1) for i=1:n, j=1:n ]

hilbert (generic function with 1 method)

In [2]:
hilbert(5)

5×5 Matrix{Rational{Int64}}:
 1//1  1//2  1//3  1//4  1//5
 1//2  1//3  1//4  1//5  1//6
 1//3  1//4  1//5  1//6  1//7
 1//4  1//5  1//6  1//7  1//8
 1//5  1//6  1//7  1//8  1//9

V tomto notebooku si ukážeme, jak (například) výpočet inverze této matice probíhá, resp. selhává, v různých aritmetikách a různých rozměrů.

Použijeme naší implementaci, ale podíváme se i na metodu dostupnou v Julia.

In [3]:
using LinearAlgebra: inv, I, norm
include("gauss.jl");

const NMAX = 15

function mybench(T, method, nrange)
    for n=nrange
        Hn = T.(hilbert(n))
        Hninv = method(Hn)
        
        println("n = ", n)
        println("Inverse" * (n > 5 ? "(a section):" : ":"))
        show(stdout, "text/plain", Hninv[1:(min(5,n)), 1:(min(5,n))])
        println("\nResidue error: ", norm(Hn * Hninv - I(n), Inf))
        println("~"^40)
    end
end

mybench (generic function with 1 method)

Matice nebudeme v zájmu přehlednosti zobrazovat celé, ale pouze $5 \times 5$ výřez vlevo nahoře.

Správnost výpočtu budeme měřit velikostí chyby součinu $H_n$ a její údajné inverze od jednotkové matice (definice inverzní matice). Chybu budeme měřit pomocí tzv. maximové normy, tedy největší hodnoty rozdílu absolutních hodnot rozdílů složek ("reziduum"). Správný výsledek je $0$. Čím větší číslo, tím horší výsledek.

## Exaktní výpočet v racionálních číslech `Rational{Int64}` v naší implementaci

In [4]:
mybench(Rational{Int64}, inverse, 1:NMAX)

n = 1
Inverse:
1×1 Matrix{Rational{Int64}}:
 1//1
Residue error: 0.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
n = 2
Inverse:
2×2 Matrix{Rational{Int64}}:
  4//1  -6//1
 -6//1  12//1
Residue error: 0.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
n = 3
Inverse:
3×3 Matrix{Rational{Int64}}:
   9//1   -36//1    30//1
 -36//1   192//1  -180//1
  30//1  -180//1   180//1
Residue error: 0.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
n = 4
Inverse:
4×4 Matrix{Rational{Int64}}:
   16//1   -120//1    240//1   -140//1
 -120//1   1200//1  -2700//1   1680//1
  240//1  -2700//1   6480//1  -4200//1
 -140//1   1680//1  -4200//1   2800//1
Residue error: 0.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
n = 5
Inverse:
5×5 Matrix{Rational{Int64}}:
    25//1    -300//1     1050//1    -1400//1     630//1
  -300//1    4800//1   -18900//1    26880//1  -12600//1
  1050//1  -18900//1    79380//1  -117600//1   56700//1
 -1400//1   26880//1  -117600//1   179200//1  -88200//1
   630//1  -12600//1    56700//1   -88200//1   4

LoadError: OverflowError: 208 * 54674698473158400 overflowed for type Int64

Co vidíme? Prvky inverzní matice jsou velká celá čísla. Výseledek je přesný. Jediným nebezpečím zde by bylo přetečení rozsahu `Int64`. K čemuž došlo už pro `n = 13`.
Toto bychom ale mohli ošetřit změnou typu prvků matice na `Rational{BigInt}`.

## Exaktní výpočet v racionálních číslech `Rational{BigInt}` v naší implementaci

In [5]:
mybench(Rational{BigInt}, inverse, 1:NMAX)

n = 1
Inverse:
1×1 Matrix{Rational{BigInt}}:
 1//1
Residue error: 0.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
n = 2
Inverse:
2×2 Matrix{Rational{BigInt}}:
  4//1  -6//1
 -6//1  12//1
Residue error: 0.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
n = 3
Inverse:
3×3 Matrix{Rational{BigInt}}:
   9//1   -36//1    30//1
 -36//1   192//1  -180//1
  30//1  -180//1   180//1
Residue error: 0.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
n = 4
Inverse:
4×4 Matrix{Rational{BigInt}}:
   16//1   -120//1    240//1   -140//1
 -120//1   1200//1  -2700//1   1680//1
  240//1  -2700//1   6480//1  -4200//1
 -140//1   1680//1  -4200//1   2800//1
Residue error: 0.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
n = 5
Inverse:
5×5 Matrix{Rational{BigInt}}:
    25//1    -300//1     1050//1    -1400//1     630//1
  -300//1    4800//1   -18900//1    26880//1  -12600//1
  1050//1  -18900//1    79380//1  -117600//1   56700//1
 -1400//1   26880//1  -117600//1   179200//1  -88200//1
   630//1  -12600//1    56700//1   -88200//

Zde dostáváme dobré výsledky, samozřejmě v jistý okamžik by opět došlo k přetečení. Reziduum je zde přesně nula (exaktní aritmetika).

## Nepřesný výpočet ve `Float64` v naší implementaci

In [6]:
mybench(Float64, inverse, 1:15)

n = 1
Inverse:
1×1 Matrix{Float64}:
 1.0
Residue error: 0.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
n = 2
Inverse:
2×2 Matrix{Float64}:
  4.0  -6.0
 -6.0  12.0
Residue error: 0.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
n = 3
Inverse:
3×3 Matrix{Float64}:
   9.0   -36.0    30.0
 -36.0   192.0  -180.0
  30.0  -180.0   180.0
Residue error: 1.4210854715202004e-14
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
n = 4
Inverse:
4×4 Matrix{Float64}:
   16.0   -120.0    240.0   -140.0
 -120.0   1200.0  -2700.0   1680.0
  240.0  -2700.0   6480.0  -4200.0
 -140.0   1680.0  -4200.0   2800.0
Residue error: 1.2064423534259876e-13
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
n = 5
Inverse:
5×5 Matrix{Float64}:
    25.0    -300.0     1050.0    -1400.0     630.0
  -300.0    4800.0   -18900.0    26880.0  -12600.0
  1050.0  -18900.0    79380.0  -117600.0   56700.0
 -1400.0   26880.0  -117600.0   179200.0  -88200.0
   630.0  -12600.0    56700.0   -88200.0   44100.0
Residue error: 7.905454069145948e-12
~~~~~~~~~~~~

Zde vidíme, že od cca $n=10$ už je chyba poměrně velká. Od této hodnoty se výsledku už moc nedá věřit, je špatně!

## Nepřesný výpočet v `BigFloat` v naší implementaci

In [7]:
mybench(BigFloat, inverse, 2:NMAX)

n = 2
Inverse:
2×2 Matrix{BigFloat}:
  4.0  -6.0
 -6.0  12.0
Residue error: 0.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
n = 3
Inverse:
3×3 Matrix{BigFloat}:
   9.0   -36.0    30.0
 -36.0   192.0  -180.0
  30.0  -180.0   180.0
Residue error: 1.105429575052088912049453038438451145102848046647844017283034044181579750804791e-75
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
n = 4
Inverse:
4×4 Matrix{BigFloat}:
   16.0   -120.0    240.0   -140.0
 -120.0   1200.0  -2700.0   1680.0
  240.0  -2700.0   6480.0  -4200.0
 -140.0   1680.0  -4200.0   2800.0
Residue error: 1.768687320083342259279124861501521832164556874636550427652854470690527601287665e-74
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
n = 5
Inverse:
5×5 Matrix{BigFloat}:
    25.0    -300.0     1050.0    -1400.0     630.0
  -300.0    4800.0   -18900.0    26880.0  -12600.0
  1050.0  -18900.0    79380.0  -117600.0   56700.0
 -1400.0   26880.0  -117600.0   179200.0  -88200.0
   630.0  -12600.0    56700.0   -88200.0   44100.0
Residue error: 1.13195

V této aritmetice pro takto malé rozměry nemáme problém. Numerický problém se ale začne projevovat později. Artimetika v `BigFloat` není exaktní.

In [8]:
mybench(BigFloat, inverse, 42:47)

n = 42
Inverse(a section):
5×5 Matrix{BigFloat}:
 1764.0          -1.55497e+06   4.56123e+08  -6.6708e+10    5.83028e+12
   -1.55497e+06   1.8276e+09   -6.03109e+11   9.4085e+13   -8.56566e+15
    4.56123e+08  -6.03109e+11   2.12294e+14  -3.44978e+16   3.23048e+18
   -6.6708e+10    9.4085e+13   -3.44978e+16   5.76607e+18  -5.512e+20
    5.83028e+12  -8.56566e+15   3.23048e+18  -5.512e+20     5.35276e+22
Residue error: 3.9680992199870308595561151608632766141226966283284127712249755859375e-09
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
n = 43
Inverse(a section):
5×5 Matrix{BigFloat}:
 1849.0          -1.70848e+06   5.25356e+08  -8.05546e+10   7.38283e+12
   -1.70848e+06   2.10484e+09  -7.28144e+11   1.19092e+14  -1.13696e+16
    5.25356e+08  -7.28144e+11   2.68685e+14  -4.5776e+16    4.49504e+18
   -8.05546e+10   1.19092e+14  -4.5776e+16    8.0217e+18   -8.04112e+20
    7.38283e+12  -1.13696e+16   4.49504e+18  -8.04112e+20   8.18854e+22
Residue error: 5.5565031708315355632747580688635125056

Samozřejmě bychom mohli zvětšit přesnost, ale vždy na tento problém narazíme.

## Nepřesný výpočet ve `Float64` pomocí Julia metody `inv`

Metoda `inv` vždy rovnou argument "shodí" do strojových čísel, proto se zde podíváme jen na `Float64` případ.

In [9]:
mybench(Float64, inv, 1:NMAX)

n = 1
Inverse:
1×1 Matrix{Float64}:
 1.0
Residue error: 0.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
n = 2
Inverse:
2×2 Matrix{Float64}:
  4.0  -6.0
 -6.0  12.0
Residue error: 0.0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
n = 3
Inverse:
3×3 Matrix{Float64}:
   9.0   -36.0    30.0
 -36.0   192.0  -180.0
  30.0  -180.0   180.0
Residue error: 7.105427357601002e-15
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
n = 4
Inverse:
4×4 Matrix{Float64}:
   16.0   -120.0    240.0   -140.0
 -120.0   1200.0  -2700.0   1680.0
  240.0  -2700.0   6480.0  -4200.0
 -140.0   1680.0  -4200.0   2800.0
Residue error: 2.2737367544323206e-13
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
n = 5
Inverse:
5×5 Matrix{Float64}:
    25.0    -300.0     1050.0    -1400.0     630.0
  -300.0    4800.0   -18900.0    26880.0  -12600.0
  1050.0  -18900.0    79380.0  -117600.0   56700.0
 -1400.0   26880.0  -117600.0   179200.0  -88200.0
   630.0  -12600.0    56700.0   -88200.0   44100.0
Residue error: 2.3759039180502434e-11
~~~~~~~~~~~~

Vidíme, že `inv` se v tomto případě  chová hodně "podobě špatně" jako naše implementace.